In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import config
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer

In [2]:
def load_image_dataset(data_dir, img_size=(config.image_size, config.image_size)):
    """
    Загружает датасет изображений из директории, организованной по папкам.

    Args:
        data_dir: Путь к директории с данными (dataset).
        img_size: Размер, до которого будут изменены изображения.

    Returns:
        Кортеж (images, labels, class_names), где images - массив NumPy с изображениями, 
        labels - массив NumPy с метками классов (целочисленными),
        class_names - список названий классов (строки).  Возвращает None, если возникла ошибка.
    """
    images = []
    labels = []
    class_names = []

    try:
        for class_name in os.listdir(data_dir):
            class_dir = os.path.join(data_dir, class_name)
            if os.path.isdir(class_dir):
                class_names.append(class_name)
                for filename in tqdm(os.listdir(class_dir), desc=f"Загрузка класса {class_name}"):
                    filepath = os.path.join(class_dir, filename)
                    try:
                        img = cv2.imread(filepath)
                        if img is not None:
                            img = cv2.resize(img, img_size)
                            images.append(img)
                            labels.append(class_name)
                    except Exception as e:
                        print(f"Ошибка при загрузке изображения {filepath}: {e}")

        # Преобразование меток в числовые значения (one-hot encoding не нужен в данном случае, если используем sparse_categorical_crossentropy)
        label_encoder = {class_name: i for i, class_name in enumerate(class_names)}
        numerical_labels = [label_encoder[label] for label in labels]

        return np.array(images), np.array(numerical_labels), class_names

    except FileNotFoundError:
        print(f"Ошибка: Директория {data_dir} не найдена.")
        return None
    except Exception as e:
        print(f"Произошла неизвестная ошибка: {e}")
        return None


In [3]:
if __name__=="__main__":
    debug = False
    result = load_image_dataset(config.data_dir)
    if result:
        images, labels, class_names = result

Загрузка класса fogsmog: 100%|██████████| 851/851 [00:02<00:00, 327.20it/s]


In [4]:
images_flat = images.reshape(images.shape[0], -1)
lb = LabelBinarizer()
y = lb.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(images_flat, y, test_size=0.2, random_state=42)

In [5]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [6]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Точность:", accuracy)

Точность: 0.44752186588921283


In [8]:
image_dir = "./TRAIN_DATASET"
image_paths = []
for root, directories, files in os.walk(image_dir):
    for name in files:
        if name.endswith(".jpg") or name.endswith(".png"):
            image_paths.append(os.path.join(root, name))

# Загрузка изображений
images = []
for image_path in image_paths:
    image = cv2.imread(image_path)
    if image is not None:
        images.append(image)

# Изменение размера изображений
img_size = (config.image_size, config.image_size)
for i in range(len(images)):
    images[i] = cv2.resize(images[i], img_size)

# Преобразование изображений в одномерные массивы
images_flat = []
for image in images:
    image_flat = image.reshape(-1)
    images_flat.append(image_flat)

# Выполнение предсказаний
predictions = []
for image_flat in images_flat:
    image_flat = np.expand_dims(image_flat, axis=0)
    prediction = clf.predict(image_flat)
    predictions.append(prediction)


# Получение классов погоды
weather_classes = []
for prediction in predictions:
    weather_class = lb.classes_[np.argmax(prediction)]
    weather_classes.append(weather_class)

# Печать предсказанных классов погоды
for i in range(len(weather_classes)):
    print(f"Предсказанный класс погоды для изображения {image_paths[i]}: {class_names[weather_classes[i]]}")

Предсказанный класс погоды для изображения ./TRAIN_DATASET/2908.jpg: sandstorm
Предсказанный класс погоды для изображения ./TRAIN_DATASET/0594.jpg: rainbow
Предсказанный класс погоды для изображения ./TRAIN_DATASET/6090.jpg: glaze
Предсказанный класс погоды для изображения ./TRAIN_DATASET/0840.jpg: snow
Предсказанный класс погоды для изображения ./TRAIN_DATASET/4084.jpg: fogsmog
